In [2]:
import pandas as pd
import numpy as np
import evaluate
import torch
from sklearn.model_selection import train_test_split
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from datasets import Dataset

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:

class model:
    def __init__(self, df, num_labels, model_id='bert-base-cased', ckpt_dir='ckpts', num_epochs=3):
        """
        General Model Class for task

        Args:
            df (pd.DataFrame): {"text":str, "label":int}
            num_labels (int): number of labels in the task
            model_id (str, optional): model_id of HuggingFace model to use. Defaults to "bert-base-cased".
            ckpt_dir (str, optional): checkpoint dir to store the checkpoints. Defaults to 'ckpts'.
            num_epochs (int, optional): number of epochs to train the model. Defaults to 3.
        """
        self.model_id = model_id
        self.df = df
        self.num_labels = num_labels
        self.ckpt_dir = ckpt_dir
        self.num_epochs = num_epochs
        self.model = AutoModelForSequenceClassification.from_pretrained(self.model_id, num_labels=self.num_labels)
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_id)
        self.training_args = TrainingArguments(
            output_dir=self.ckpt_dir, 
            evaluation_strategy="epoch", 
            num_train_epochs=self.num_epochs,
        )
        self.metric = evaluate.load("f1")

    def tokenize_function(self, data):
        return self.tokenizer(data["text"], padding="max_length", truncation=True, return_tensors='pt')

    def compute_metrics(self, eval_pred):
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        return self.metric.compute(predictions=predictions, references=labels, average="macro")

    def train(self):
        X_train, X_eval, y_train, y_eval = train_test_split(self.df['text'], self.df['label'], test_size=0.1, random_state=42)
        train_data = Dataset.from_dict({
            'text': X_train.values,
            'label': y_train.values
        })
        eval_data = Dataset.from_dict({
            'text': X_eval.values,
            'label': y_eval.values
        })
        tokenized_train_dataset = train_data.map(self.tokenize_function, batched=True)
        tokenized_eval_dataset = eval_data.map(self.tokenize_function, batched=True)
        trainer = Trainer(
            model=self.model,
            args=self.training_args,
            train_dataset=tokenized_train_dataset,
            eval_dataset=tokenized_eval_dataset,
            compute_metrics=self.compute_metrics,
        )   
        result = trainer.train()
        return result

In [4]:
def binary_baseline(df):
    df.drop(['rewire_id', 'label_category', 'label_vector'], axis=1, inplace=True)    
    df['label_sexist'].replace({'not sexist':0, 'sexist':1}, inplace=True)
    df.rename(columns={'label_sexist':'label'}, inplace=True)
    print(df.head(10))
    obj = model(df, 2)
    result = obj.train()
    


In [5]:
df = pd.read_csv('data/train_all_tasks.csv')
binary_baseline(df)

                                                text  label
0              Damn, this writing was pretty chaotic      0
1  Yeah, and apparently a bunch of misogynistic v...      0
2       How the FUCK is this woman still an MP!!!???      0
3  Understand. Know you're right. At same time I ...      0
4     Surprized they didn't stop and rape some women      0
5  yeah but in order to keep the benefit i have t...      0
6  Hahaha I just wish they would live their truth...      0
7  head mod said she talked to him after he made ...      0
8  just the inside of your wallet: cash, bank car...      1
9  We most definitely did. I didn’t go too much i...      0


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

  0%|          | 0/13 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/home/users/jtripathy/.local/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 12600
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 4725
  Number of trainable parameters = 108311810


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 